In [2]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики

from sklearn.model_selection import train_test_split #сплитование выборки

from sklearn.model_selection import cross_val_score

import optuna

%matplotlib inline

from colorama import Fore, Back, Style
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.3f}'.format)

import warnings
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv('data/biological_response.csv')

In [4]:
data.head(4)

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000,0.497,0.100,0.000,0.133,0.678,0.273,0.585,0.744,...,0,0,0,0,0,0,0,0,0,0
1,1,0.367,0.606,0.050,0.000,0.111,0.803,0.106,0.412,0.837,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033,0.480,0.000,0.000,0.210,0.610,0.356,0.518,0.679,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000,0.539,0.000,0.500,0.196,0.724,0.236,0.289,0.805,...,0,0,0,0,0,0,0,0,0,0


In [5]:
data.shape

(3751, 1777)

In [6]:
# Подготовка данных для следующей работы
X = data.drop(['Activity'], axis=1)
y = data['Activity']

X_train, X_test, y_train, y_test = train_test_split(X, y,\
  stratify=y, random_state = 1, test_size = 0.2)

In [7]:
# В данную переменную буду сохранять результаты моделей [f1_train, f1_test, название]
f1_score_results = []
# Константы часто используемых параметров
RANDOM_STATE = 42
MAX_ITER = 500

In [8]:

def printAndSaveModelResults(model, methodName):
  """функция для вывода результатов score для train и test 
     и также для сохранения этих значений для последующего использования
  Args:
      model (model): Ссылка на обучепнную модель
      methodName (string): Название примененной модели
  """
  y_train_pred = model.predict(X_train)
  metrics_train = metrics.f1_score(y_train, y_train_pred).round(6)
  print(f"{Fore.LIGHTBLUE_EX}f1_score на тренировочном наборе: {Fore.LIGHTGREEN_EX}{metrics_train:.3f}{Fore.RESET}")
  y_test_pred = model.predict(X_test)
  metrics_test = metrics.f1_score(y_test, y_test_pred).round(6)
  print(f"{Fore.LIGHTBLUE_EX}f1_score на тестовом наборе: {Fore.LIGHTGREEN_EX}{metrics_test:.3f}{Fore.RESET}")

  f1_score_results.append({'result_train': metrics_train,
                           'result_test': metrics_test,
                          'method': methodName})

# Моделирование

## Логистическая регрессия

Зафиксируем только метрики, которые были получены без дополнительной настройки, т.е со значениями гиперпараметров, установленных по умолчанию:

In [9]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(max_iter = MAX_ITER)
#Обучаем модель, минимизируя logloss
log_reg.fit(X_train, y_train)

printAndSaveModelResults(log_reg,'LogisticRegression native')



f1_score на тренировочном наборе: 0.885
f1_score на тестовом наборе: 0.781


## Случайный лес

In [10]:
#Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(random_state=RANDOM_STATE)

#Обучаем модель
rf.fit(X_train, y_train)

printAndSaveModelResults(rf,'RandomForestClassifier native')

f1_score на тренировочном наборе: 1.000
f1_score на тестовом наборе: 0.813


## Optuna



In [11]:
scoreCalc = 'standard'
modelType = 'logReg'

In [114]:
def optuna_optimize(trial):
  
  # задаем пространства поиска гиперпараметров
  if modelType == 'logReg':
    model = linear_model.LogisticRegression(
      penalty = trial.suggest_categorical('penalty', ['l2', 'none']),
      solver = trial.suggest_categorical('solver',['lbfgs', 'sag']),
      C = trial.suggest_float('C', 0.1, 1, step = 0.1),
      random_state = RANDOM_STATE,
      n_jobs=-1
    )

  elif modelType == 'ranFor':
    n_estimators = trial.suggest_int('n_estimators', 100, 310, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 3, 7, 1)
    max_depth = trial.suggest_int('max_depth', 15, 40, 1)

    # создаем модель
    model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=RANDOM_STATE,
                                          n_jobs=-1
                                          )
  # обучаем модель
  if scoreCalc == 'standard':
    model.fit(X_train, y_train)
    score = metrics.f1_score(y_test, model.predict(X_test))
  elif scoreCalc == 'cross':
      score = cross_val_score(model, X_train, y_train, cv = 5, scoring="f1", n_jobs = -1).mean()  
  return score

#### Логистическая регрессия

##### Стандартный расчет score

In [115]:
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study_logreg = optuna.create_study(study_name="LogisticRegression Optuna Optimization", 
                                   direction="maximize",
                                  #  storage='sqlite:///ex.db',
                                  #  load_if_exists=True
                                   )
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_logreg.optimize(optuna_optimize, n_trials=10
                      , n_jobs=-1
                      )

# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study_logreg.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study_logreg.best_value))

[I 2023-04-28 17:21:07,398] Using an existing study with name 'LogisticRegression Optuna Optimization' instead of creating a new one.
[I 2023-04-28 17:21:13,679] Trial 141 finished with value: 0.804337281737858 and parameters: {'n_estimators': 150, 'min_samples_leaf': 3, 'max_depth': 21}. Best is trial 141 with value: 0.804337281737858.
[I 2023-04-28 17:21:14,906] Trial 142 finished with value: 0.8008035094272052 and parameters: {'n_estimators': 180, 'min_samples_leaf': 7, 'max_depth': 29}. Best is trial 141 with value: 0.804337281737858.
[I 2023-04-28 17:21:17,854] Trial 138 finished with value: 0.8005132284290231 and parameters: {'n_estimators': 190, 'min_samples_leaf': 6, 'max_depth': 23}. Best is trial 141 with value: 0.804337281737858.
[I 2023-04-28 17:21:18,244] Trial 136 finished with value: 0.8006739006971187 and parameters: {'n_estimators': 120, 'min_samples_leaf': 7, 'max_depth': 33}. Best is trial 141 with value: 0.804337281737858.
[I 2023-04-28 17:21:18,458] Trial 140 finis

Наилучшие значения гиперпараметров {'max_depth': 26, 'min_samples_leaf': 4, 'n_estimators': 310}
f1_score на обучающем наборе: 0.81


In [13]:
# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(**study_logreg.best_params,random_state=RANDOM_STATE, )
model.fit(X_train, y_train)

printAndSaveModelResults(model,'LogisticRegression Optuna Optimization')

f1_score на тренировочном наборе: 0.849
f1_score на тестовом наборе: 0.790


##### Кросс-валидация

In [14]:
scoreCalc = "cross"
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study_logreg_cross = optuna.create_study(study_name="LogisticRegression Optuna Optimization with Cross Validation",
                                         direction="maximize",
                                         storage='sqlite:///ex.db',
                                         load_if_exists=True,
                                         )
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_logreg_cross.optimize(optuna_optimize, n_trials=20, n_jobs=-1)

# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study_logreg_cross.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study_logreg_cross.best_value))

[I 2023-04-28 15:11:30,405] A new study created in RDB with name: LogisticRegression Optuna Optimization with Cross Validation
[I 2023-04-28 15:11:42,677] Trial 0 finished with value: 0.7475536741581407 and parameters: {'penalty': 'none', 'solver': 'lbfgs', 'C': 0.4}. Best is trial 0 with value: 0.7475536741581407.
[I 2023-04-28 15:11:42,897] Trial 11 finished with value: 0.7738912160062705 and parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.5}. Best is trial 11 with value: 0.7738912160062705.
[I 2023-04-28 15:11:43,663] Trial 10 finished with value: 0.7738912160062705 and parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.5}. Best is trial 11 with value: 0.7738912160062705.
[I 2023-04-28 15:11:43,771] Trial 6 finished with value: 0.7794662578389194 and parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.30000000000000004}. Best is trial 6 with value: 0.7794662578389194.
[I 2023-04-28 15:11:43,816] Trial 14 finished with value: 0.7475536741581407 and parameters: {'pena

Наилучшие значения гиперпараметров {'C': 0.1, 'penalty': 'l2', 'solver': 'sag'}
f1_score на обучающем наборе: 0.78


In [15]:
# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(**study_logreg_cross.best_params,random_state=RANDOM_STATE, )
model.fit(X_train, y_train)

printAndSaveModelResults(model,'LogisticRegression Optuna Optimization with Cross Validation')

f1_score на тренировочном наборе: 0.849
f1_score на тестовом наборе: 0.790


#### Случайный лес

##### Стандартный расчет score

In [17]:

scoreCalc = "standard"
modelType = "ranFor"
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study_ranFor = optuna.create_study(study_name="RandomForestClassifier Optuna Optimization",
                                   direction="maximize",
                                  #  storage='sqlite:///ex.db',
                                   load_if_exists=False,
                                   )
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_ranFor.optimize(optuna_optimize, n_trials=10)

# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study_ranFor.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study_ranFor.best_value))

[I 2023-04-28 15:12:15,510] A new study created in memory with name: RandomForestClassifier Optuna Optimization
[I 2023-04-28 15:12:15,980] Trial 0 finished with value: 0.8247914183551848 and parameters: {'n_estimators': 140, 'min_samples_leaf': 4, 'max_depth': 35}. Best is trial 0 with value: 0.8247914183551848.
[I 2023-04-28 15:12:16,717] Trial 1 finished with value: 0.8260355029585799 and parameters: {'n_estimators': 270, 'min_samples_leaf': 3, 'max_depth': 17}. Best is trial 1 with value: 0.8260355029585799.
[I 2023-04-28 15:12:17,470] Trial 2 finished with value: 0.8274231678486997 and parameters: {'n_estimators': 290, 'min_samples_leaf': 4, 'max_depth': 32}. Best is trial 2 with value: 0.8274231678486997.
[I 2023-04-28 15:12:17,985] Trial 3 finished with value: 0.8299643281807373 and parameters: {'n_estimators': 170, 'min_samples_leaf': 3, 'max_depth': 33}. Best is trial 3 with value: 0.8299643281807373.
[I 2023-04-28 15:12:18,516] Trial 4 finished with value: 0.8233890214797136 

Наилучшие значения гиперпараметров {'n_estimators': 170, 'min_samples_leaf': 3, 'max_depth': 33}
f1_score на обучающем наборе: 0.83


In [18]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study_ranFor.best_params,random_state=RANDOM_STATE, )
model.fit(X_train, y_train)

printAndSaveModelResults(model,'RandomForestClassifier Optuna Optimization')

f1_score на тренировочном наборе: 0.978
f1_score на тестовом наборе: 0.830


##### Кросс-валидация

In [19]:
scoreCalc = "cross"

# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study_ranFor_cross = optuna.create_study(study_name="RandomForestClassifier Optuna Optimization with Cross Validation",
                                   direction="maximize",
                              #      storage='sqlite:///ex.db',
                              #      load_if_exists=True,
                                         )
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_ranFor_cross.optimize(optuna_optimize, n_trials=10, n_jobs=-1)

# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study_ranFor_cross.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study_ranFor_cross.best_value))

[I 2023-04-28 15:12:31,956] A new study created in memory with name: RandomForestClassifier Optuna Optimization with Cross Validation
[I 2023-04-28 15:12:33,481] Trial 2 finished with value: 0.8068020282959651 and parameters: {'n_estimators': 120, 'min_samples_leaf': 3, 'max_depth': 24}. Best is trial 2 with value: 0.8068020282959651.
[I 2023-04-28 15:12:37,039] Trial 4 finished with value: 0.8001900224462137 and parameters: {'n_estimators': 150, 'min_samples_leaf': 6, 'max_depth': 29}. Best is trial 2 with value: 0.8068020282959651.
[I 2023-04-28 15:12:37,238] Trial 7 finished with value: 0.8020140244506146 and parameters: {'n_estimators': 200, 'min_samples_leaf': 6, 'max_depth': 28}. Best is trial 2 with value: 0.8068020282959651.
[I 2023-04-28 15:12:37,919] Trial 1 finished with value: 0.8051580116960169 and parameters: {'n_estimators': 110, 'min_samples_leaf': 4, 'max_depth': 31}. Best is trial 2 with value: 0.8068020282959651.
[I 2023-04-28 15:12:37,960] Trial 0 finished with valu

Наилучшие значения гиперпараметров {'n_estimators': 250, 'min_samples_leaf': 3, 'max_depth': 38}
f1_score на обучающем наборе: 0.81


In [20]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study_ranFor_cross.best_params,random_state=RANDOM_STATE, )
model.fit(X_train, y_train)

printAndSaveModelResults(model,'RandomForestClassifier Optuna Optimization with Cross Validation')

f1_score на тренировочном наборе: 0.977
f1_score на тестовом наборе: 0.826


In [21]:
optuna.visualization.is_available()

True

In [22]:
optuna.visualization.plot_optimization_history(study_ranFor, target_name="f1_score")

In [23]:
optuna.visualization.plot_param_importances(study_ranFor, target_name="f1_score")

In [ ]:
optuna.visualization.plot_contour(study_ranFor, params=["max_depth", "min_samples_leaf"],
                                  target_name="f1_score")

In [24]:
resultf1ScoreData = pd.DataFrame(f1_score_results).set_index('method').sort_values(by='result_test',ascending=False)

display(resultf1ScoreData.\
        style.background_gradient(\
        axis=0, cmap='GnBu',
        low = 0.8,
        high=0.9,
        vmin = 0.8,
        vmax = 1))

,result_train,result_test
method,,
RandomForestClassifier Optuna Optimization,0.977669,0.829964
RandomForestClassifier Optuna Optimization with Cross Validation,0.977356,0.826347
RandomForestClassifier native,1.000000,0.812576
LogisticRegression Optuna Optimization,0.849348,0.790419
LogisticRegression Optuna Optimization with Cross Validation,0.849348,0.790419
LogisticRegression native,0.885226,0.780838
